In [67]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["nutriwiser_db"]
collection = db["additifs_scraped"]

# Obtenir toutes les valeurs distinctes d'un champ
distinct_names = collection.distinct("danger")

print(distinct_names)

['', 'Douteux', 'Ne pas abuser', 'Pas/peu toxique', 'Toxique', 'Très toxique']


In [68]:
distinct_toxicite_levels = distinct_names

In [87]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="nutriwiser",
    password="nutriwiser",
    database="nutriwiser_db"
)

cursor = conn.cursor()

query = "INSERT INTO toxicite (label,niveau) VALUES (%s, %s)"

distinct_toxicite_levels_dicts = [
        {
        "label": "Inconnu",
        "niveau": 0
    },
    {
        "label": "Pas/peu toxique",
        "niveau": 1
    },
    {
        "label": "Ne pas abuser",
        "niveau": 2
    },
    {
        "label": "Douteux",
        "niveau": 3
    },
    {
        "label": "Toxique",
        "niveau": 4
    },
    {
        "label": "Très toxique",
        "niveau": 5
    }
]

for toxicite_level in distinct_toxicite_levels_dicts:
    cursor.execute(query, (toxicite_level["label"], toxicite_level["niveau"]))

conn.commit()
cursor.close()
conn.close()


In [83]:
conn.rollback()

MySQLInterfaceError: Lost connection to MySQL server during query

In [88]:

filter={}

result = client['nutriwiser_db']['additifs_scraped'].find(
  filter=filter,
)

conn = mysql.connector.connect(
    host="localhost",
    user="nutriwiser",
    password="nutriwiser",
    database="nutriwiser_db"
)

cursor = conn.cursor()

# print(list(result))

for doc in result:
    try:
        code = doc.get('additive_code')
        print(code)

        names = doc.get('names') or []
        label = doc.get('danger', '').strip()
        description = doc.get('description')
        description_avancee = doc.get('description_avancee')
        remarques = doc.get('remarques')

        # Mapping label → niveau
        niveau = {
            "Pas/peu toxique": 2,
            "Ne pas abuser": 3,
            "Douteux": 4,
            "Toxique": 5,
            "Très toxique": 6
        }.get(label, 1)  # Par défaut : 1 = inconnu

        print(f"Toxicité: {niveau}")

        # Vérifie si toxicite_id existe
        cursor.execute("SELECT id FROM toxicite WHERE id = %s", (niveau,))
        if cursor.fetchone() is None:
            print(f"[ERREUR] toxicite_id {niveau} absent → skip {code}")
            continue  # <-- ne pas bloquer toute la boucle

        # Insertion dans additifs
        cursor.execute("""
            INSERT INTO additifs (code, toxicite_id, description, description_avancee, remarques)
            VALUES (%s, %s, %s, %s, %s)
        """, (code, niveau, description, description_avancee, remarques))

        # Insertion dans additifs_noms
        for name in names:
            cursor.execute("INSERT INTO additifs_noms (code_additif, nom) VALUES (%s, %s)", (code, name))

        conn.commit()
        print(f"[OK] Additif inséré : {code}")

    except mysql.connector.Error as err:
        print(f"[ERREUR MySQL] sur {code} :", err)
        conn.rollback()


E100
Toxicité: 3
[OK] Additif inséré : E100
E101
Toxicité: 3
[OK] Additif inséré : E101
E102
Toxicité: 6
[OK] Additif inséré : E102
E104
Toxicité: 6
[OK] Additif inséré : E104
E105
Toxicité: 6
[OK] Additif inséré : E105
E107
Toxicité: 5
[OK] Additif inséré : E107
E110
Toxicité: 6
[OK] Additif inséré : E110
E111
Toxicité: 6
[OK] Additif inséré : E111
E120
Toxicité: 4
[OK] Additif inséré : E120
E121
Toxicité: 6
[OK] Additif inséré : E121
E122
Toxicité: 6
[OK] Additif inséré : E122
E123
Toxicité: 6
[OK] Additif inséré : E123
E124
Toxicité: 6
[OK] Additif inséré : E124
E125
Toxicité: 6
[OK] Additif inséré : E125
E126
Toxicité: 6
[OK] Additif inséré : E126
E127
Toxicité: 6
[OK] Additif inséré : E127
E128
Toxicité: 6
[OK] Additif inséré : E128
E129
Toxicité: 6
[OK] Additif inséré : E129
E130
Toxicité: 6
[OK] Additif inséré : E130
E131
Toxicité: 5
[OK] Additif inséré : E131
E132
Toxicité: 6
[OK] Additif inséré : E132
E133
Toxicité: 6
[OK] Additif inséré : E133
E140
Toxicité: 3
[OK] Additif in

In [32]:

filter={}

result = client['nutriwiser_db']['additifs_scraped'].find(
  filter=filter,
  limit=1
)


for doc in result:
    
    Code = doc.get('code')
    
    niveau = doc.get('danger')
    
    # if Code:
    #     cursor.execute("UPDATE additifs SET toxicite = %s WHERE code = %s", (niveau, Code))
